#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import BigQuery library


In [3]:
import os
from google.cloud import bigquery

#### 3. Create a connection to BigQuery

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/MarLizana/Desktop/IH/ironhack_service_account_big_query.json"
client = bigquery.Client()

#### 4. Import data from the users table from the stackoverflow public database for United Kingdom only

In [5]:
user_data = """
SELECT 
    *
FROM 
    `bigquery-public-data.stackoverflow.users` 
    
WHERE
    location LIKE "%United Kingdom%"
    
"""

query_job = client.query(query=user_data)
user_data = query_job.to_dataframe()
user_data.head()

,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,1074,Javache,<p>Hi.</p>\n\n<p>I'm an iOS engineer at Facebo...,None,2008-08-12 10:11:09.847000+00:00,2019-10-21 15:17:18.803000+00:00,"London, United Kingdom",3107,158,21,266,None,http://thinkjavache.be/blog
1,1077,smh,<p>\nEmbedded software engineer.</p>,None,2008-08-12 10:34:56.447000+00:00,2019-09-07 17:19:22.130000+00:00,"Southampton, United Kingdom",1075,148,4,69,None,http://www.bevois.com
2,4397,andygeers,None,None,2008-09-03 13:48:30.233000+00:00,2019-11-29 12:40:30.020000+00:00,"London, United Kingdom",6057,511,12,593,None,http://www.geero.net/
3,5337,J Hunt,<p></p>,None,2008-09-09 09:07:41.840000+00:00,2015-02-19 16:29:45.007000+00:00,United Kingdom,602,1,0,127,None,None
4,6910,Jonathan,<ul><li>Started programming with Visual Basic ...,None,2008-09-15 12:48:35.843000+00:00,2019-11-29 15:15:39.270000+00:00,United Kingdom,21315,897,8,831,https://www.gravatar.com/avatar/592695fec5a951...,http://www.toolan.net


#### 5. Rename id column to user_id

In [6]:
user_data = user_data.rename(columns={'id':'user_id'})
user_data.head()

,user_id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,1074,Javache,<p>Hi.</p>\n\n<p>I'm an iOS engineer at Facebo...,None,2008-08-12 10:11:09.847000+00:00,2019-10-21 15:17:18.803000+00:00,"London, United Kingdom",3107,158,21,266,None,http://thinkjavache.be/blog
1,1077,smh,<p>\nEmbedded software engineer.</p>,None,2008-08-12 10:34:56.447000+00:00,2019-09-07 17:19:22.130000+00:00,"Southampton, United Kingdom",1075,148,4,69,None,http://www.bevois.com
2,4397,andygeers,None,None,2008-09-03 13:48:30.233000+00:00,2019-11-29 12:40:30.020000+00:00,"London, United Kingdom",6057,511,12,593,None,http://www.geero.net/
3,5337,J Hunt,<p></p>,None,2008-09-09 09:07:41.840000+00:00,2015-02-19 16:29:45.007000+00:00,United Kingdom,602,1,0,127,None,None
4,6910,Jonathan,<ul><li>Started programming with Visual Basic ...,None,2008-09-15 12:48:35.843000+00:00,2019-11-29 15:15:39.270000+00:00,United Kingdom,21315,897,8,831,https://www.gravatar.com/avatar/592695fec5a951...,http://www.toolan.net


#### 6. Import from the stackoverflow_posts table all items which have up to 20 view counts and only select 'id', 'owner_user_id', 'score', 'view_count', 'comment_count', 'favorite_count'

In [7]:
post_data = """
SELECT 
    id,
    owner_user_id,
    score,
    view_count,
    comment_count,
    favorite_count
    
FROM 
    `bigquery-public-data.stackoverflow.stackoverflow_posts`
    
WHERE

    view_count < 20;
    
"""

In [8]:
query_job = client.query(query=post_data)

In [9]:
post_data = query_job.to_dataframe()

In [10]:
post_data.head()

,id,owner_user_id,score,view_count,comment_count,favorite_count
0,35865779,NaN,0,15,1,NaN
1,35501106,NaN,0,16,0,0.0
2,37129340,1267466.0,0,15,6,NaN
3,29371904,2209530.0,0,17,2,NaN
4,36045693,NaN,1,17,0,NaN


#### 7. Rename id column to post_id and owner_user_id to user_id

In [11]:
post_data = post_data.rename(columns={'id':'post_id',
                                     'owner_user_id':'user_id'})
post_data.head()

,post_id,user_id,score,view_count,comment_count,favorite_count
0,35865779,NaN,0,15,1,NaN
1,35501106,NaN,0,16,0,0.0
2,37129340,1267466.0,0,15,6,NaN
3,29371904,2209530.0,0,17,2,NaN
4,36045693,NaN,1,17,0,NaN


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [12]:
column_order_users = ['user_id','reputation','views','up_votes',
                'down_votes']

column_order_posts = ['post_id','score', 'user_id', 'view_count',
                     'comment_count']

In [13]:
user_data = user_data[column_order_users]
post_data = post_data[column_order_posts]

In [14]:
user_data.head()

,user_id,reputation,views,up_votes,down_votes
0,1074,3107,266,158,21
1,1077,1075,69,148,4
2,4397,6057,593,511,12
3,5337,602,127,1,0
4,6910,21315,831,897,8


In [15]:
user_data.shape

(66832, 5)

In [16]:
post_data.head()

,post_id,score,user_id,view_count,comment_count
0,35865779,0,NaN,15,1
1,35501106,0,NaN,16,0
2,37129340,0,1267466.0,15,6
3,29371904,0,2209530.0,17,2
4,36045693,1,NaN,17,0


In [17]:
post_data.shape

(284796, 5)

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [18]:
merged_DF = pd.merge(user_data, post_data, on = "user_id")
merged_DF.head(10)

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,33682,3218,448,124,3,31383344,0,11,0
1,148381,9331,800,509,158,34420509,0,19,0
2,385390,717,91,15,2,37756554,0,9,0
3,1010198,412,141,19,0,33721783,0,15,0
4,1010198,412,141,19,0,36489861,0,16,0
5,1566860,345,36,45,3,33336278,1,19,0
6,1848146,1578,54,18,1,37183419,0,19,2
7,1848146,1578,54,18,1,36011395,0,17,0
8,1848146,1578,54,18,1,36476511,0,10,0
9,2135406,415,33,80,2,37750899,0,15,0


#### 9. How many missing values do you have in your merged dataframe? 

In [19]:
merged_DF.shape

(4070, 9)

In [20]:
null_merged = merged_DF.isnull().sum()

In [21]:
null_merged [null_merged > 0]

Series([], dtype: int64)

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [57]:
perc= [.15, .50, .75, .99] 
stats = merged_DF.describe(percentiles = perc).transpose()
stats

,count,mean,std,min,15%,50%,75%,99%,max
user_id,4070.0,2.287943e+06,1.905006e+06,199.0,343204.00,1719075.0,3632206.0,6344000.79,6452203.0
reputation,4070.0,5.412067e+03,1.663058e+04,1.0,89.00,1130.5,4307.5,69219.25,279695.0
views,4070.0,4.902710e+02,1.228242e+03,0.0,18.00,137.0,418.5,6176.00,27257.0
up_votes,4070.0,3.685442e+02,7.549705e+02,0.0,4.00,89.0,352.0,3682.31,9057.0
down_votes,4070.0,3.605627e+01,2.042824e+02,0.0,0.00,3.0,16.0,745.20,8468.0
post_id,4070.0,3.428236e+07,4.262270e+06,1906903.0,31287553.95,35774363.0,37114180.0,37756559.27,37770535.0
score,4070.0,1.083538e-01,5.428813e-01,-7.0,0.00,0.0,0.0,2.00,4.0
view_count,4070.0,1.467224e+01,3.583939e+00,2.0,11.00,15.0,18.0,19.00,19.0
comment_count,4070.0,8.533170e-01,1.549656e+00,0.0,0.00,0.0,1.0,7.00,15.0


In [58]:
stats['IQR'] = stats['99%'] - stats['15%']

In [59]:
columns_outliers = ["reputation", "views", "up_votes","down_votes", "score","view_count","comment_count"]
outliers = pd.DataFrame(columns=columns_outliers)
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'15%'] - cutoff
    upper = stats.at[col,'99%'] + cutoff
    results = merged_DF[(merged_DF[col] < lower) |
                   (merged_DF[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

In [60]:
outliers

,Outlier,comment_count,down_votes,post_id,reputation,score,up_votes,user_id,view_count,views
1217,reputation,2,461,37642104.0,187688,4,3899,324584.0,18,27257
2059,reputation,0,97,33842576.0,235072,0,927,6068.0,6,10965
2060,reputation,0,97,36953621.0,235072,0,927,6068.0,11,10965
2061,reputation,0,97,37093383.0,235072,1,927,6068.0,11,10965
2062,reputation,0,97,37257134.0,235072,1,927,6068.0,18,10965
...,...,...,...,...,...,...,...,...,...,...
984,score,2,0,37263029.0,1,-4,0,6342677.0,12,3
991,score,4,0,29716312.0,3,-4,0,4804161.0,19,0
1481,score,1,0,37505897.0,29,-4,0,5089944.0,15,19
2952,score,4,0,37667739.0,43,-7,3,6118448.0,14,5


In [26]:
outliers.to_csv("outliers.csv", index = False)